<a href="https://colab.research.google.com/github/zunayedology/multimodal-sensor-fusion-ppke/blob/main/Fusion02_startwiththis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
story:
let's assume we have a LIDAR
https://www.nuscenes.org/data-annotation
https://level5.lyft.com/dataset/#data-format

WE have a lidar and a detector which should detect pedestrian/traffic signs/and free space
the detector was built by someone else and we do not know anything about its quality, but we want to (and have to) use this in practice.

We want to make the best out of it using repeated measurements.

It is imporant that we asusme that these measurements are completely independent form each other...which is not ture, but we will deal with this in practice later.

"""

import numpy as np
import gdown

## LIDAR Model

In [2]:
"""
class Model():
  #this is a  simple probabilistic model representing our sensor
  def __init__ (self):
    #constructor

  def GenerateState(self):
    #this function generates a random hidden state
    #has no return value

  def GetState(self):
    #this is a function we can not use inn practice, it reveals the hidden state. We will use this to check our result.
    #returns a number, the index of the hidden state (0,1,2)

  def Measure(self):
    #this funciton makes a measuerement with a previously set hidden state
    #returns the index of the measured state (0,1,2)

  def MeasureKnownState(self,X):
    #this function makes a measurement with a known hidden state
    #the hidden state is the imnput aprametre of the function (0,1,2)
    #returns the index of the measured state (0,1,2)
"""

url = 'https://drive.google.com/uc?id=1mi9WvjTDscb3VZe1Hdg-Iv02DwXYJNR0'
output = 'LIDARModel.py'
gdown.download(url, output, quiet=False)
import LIDARModel as Model

Downloading...
From (original): https://drive.google.com/uc?id=1mi9WvjTDscb3VZe1Hdg-Iv02DwXYJNR0
From (redirected): https://drive.google.com/uc?id=1mi9WvjTDscb3VZe1Hdg-Iv02DwXYJNR0&confirm=t&uuid=e5edeeeb-c1f2-41c7-9278-54519fe87259
To: /content/LIDARModel.py
100%|██████████| 1.09k/1.09k [00:00<00:00, 4.01MB/s]


In [3]:
M = Model.Model() #initalize this model
M.GenerateState() #generate a random hidden state

M.GetState()      #we can check the hidden state...!this is not a measurement

1

In [4]:
#let's make 10 measurements with the known hidden states
for i in range(10):
  print(M.Measure())

1
1
0
1
1
0
1
2
1
1


In [6]:
# Get the avg prediction value for a given hidden state
# calculate the avg freq

repeat = 1000
state_count = np.zeros(3)

for i in range(repeat):
    measurement = M.Measure()
    state_count[measurement] += 1

print(state_count)

state_count /= repeat
print(state_count)


[308. 585. 107.]
[0.308 0.585 0.107]


In [7]:
# get relative freq matrix

repeat = 1000

RelativeFreqMatrix = np.zeros((3,3))

for hidden_state in range(3):
    for i in range(repeat):
        actual_obs = M.MeasureKnownState(hidden_state)
        # RelativeFreqMatrix[Row, Col]
        # RelativeFreqMatrix[State, observation]
        RelativeFreqMatrix[hidden_state, actual_obs] += 1

print(RelativeFreqMatrix)


[[509. 175. 316.]
 [301. 594. 105.]
 [ 98. 106. 796.]]


In [15]:
# Observation probability, observation matrix

ObservationProbability = np.sum(RelativeFreqMatrix, 0) / np.sum(RelativeFreqMatrix)
print("Observation Probability", ObservationProbability)

# What is the probablity of being in a state if I got an observation?

ObservationMatrix = RelativeFreqMatrix / np.sum(RelativeFreqMatrix, 0)
print("Observation Matrix\n", ObservationMatrix)

Observation Probability [0.30266667 0.29166667 0.40566667]
Observation Matrix
 [[0.56057269 0.2        0.25965489]
 [0.3314978  0.67885714 0.08627773]
 [0.10792952 0.12114286 0.65406738]]


In [18]:
# make Bayesian estimation

# P(A|B) = (P(B|A) * P(A)) / P(B)
# P(hidden|measurement) = (P(measurement|hidden) * P(hidden)) / P(measured)
# P(B_measurement|A_hidden): ObseravtionMatrix
# P(A_hidden): Prior assumption: What do we know about hidden state
# P(B_measurement): ObservationProbability

M.GenerateState()
print("The hidden state: ", M.GetState())

HiddenStateDistribution = np.array([1/3, 1/3, 1/3])

prior_assumption = HiddenStateDistribution

repeat = 10
HiddenStateDistribution = np.array([1/3, 1/3, 1/3])

for i in range(repeat):
    measurement = M.Measure()
    # print(measurement)
    # print(ObservationMatrix[:, measurement])
    prediction = ObservationMatrix[:, measurement] * prior_assumption / ObservationProbability
    prediction = prediction / np.sum(prediction)
    prior_assumption = prediction
    print(prediction)


The hidden state:  1
[0.20103653 0.70811072 0.09085275]
[0.07347095 0.91152385 0.0150052 ]
[0.11570402 0.88090146 0.00339452]
[3.59325205e-02 9.63591070e-01 4.76409603e-04]
[1.04753321e-02 9.89461902e-01 6.27658930e-05]
[2.97789428e-02 9.69885716e-01 3.35340836e-04]
[8.64120126e-03 9.91314823e-01 4.39758633e-05]
[2.46865504e-03 9.97525667e-01 5.67758635e-06]
[7.02109404e-04 9.99297161e-01 7.29745150e-07]
[2.03350085e-03 9.97962527e-01 3.97220855e-06]
